<a href="https://colab.research.google.com/github/E1250/other-support-ref/blob/main/Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Youtube Video - https://www.youtube.com/watch?v=RbZDYc_Cvsk
* TF Federated Source - https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification
* Formal Colab code - https://colab.research.google.com/github/tensorflow/federated/blob/v0.61.0/docs/tutorials/federated_learning_for_image_classification.ipynb

In [ ]:
!pip install --upgrade tensorflow-federated

In [1]:
import collections
import numpy as np
np.random.seed(0)

import tensorflow as tf
from tensorflow.python.keras.optimizer_v2 import gradient_descent

import tensorflow_federated as tff
from random import choices

tff.federated_computation(lambda: 'Hello, World!')()

tf.compat.v1.enable_v2_behavior()

In [2]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

In [4]:
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

In [5]:
def preprocess(dataset):
  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1]),
        y=tf.reshape(element['label'], [-1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [6]:
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]

In [13]:
sample_clients = emnist_train.client_ids[0:NUM_CLIENTS]

federated_train_data = make_federated_data(emnist_train, sample_clients)
print(f'Number of client datasets: {len(federated_train_data)}')
print(f'First dataset: {federated_train_data[0]}')

federated_test_data = make_federated_data(emnist_test, sample_clients)

Number of client datasets: 10
First dataset: <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))])>


In [18]:
sample_batch = iter(federated_train_data[0]).next()
sample_batch = collections.OrderedDict(
    x = sample_batch['x'].numpy(),
    y = sample_batch['y'].numpy()
)

In [19]:
sample_batch

OrderedDict([('x',
              array([[1., 1., 1., ..., 1., 1., 1.],
                     [1., 1., 1., ..., 1., 1., 1.],
                     [1., 1., 1., ..., 1., 1., 1.],
                     ...,
                     [1., 1., 1., ..., 1., 1., 1.],
                     [1., 1., 1., ..., 1., 1., 1.],
                     [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)),
             ('y',
              array([[2],
                     [1],
                     [5],
                     [7],
                     [1],
                     [7],
                     [7],
                     [1],
                     [4],
                     [7],
                     [4],
                     [2],
                     [2],
                     [5],
                     [4],
                     [1],
                     [1],
                     [0],
                     [0],
                     [9]], dtype=int32))])